# Построение и обучение модели многоклассовой классификации на основе LSTM

Как вы, наверное, заметили, ещё на этапе сбора и подготовки данных я проектировал датасет под обучение с помощью инструментов, которые даёт платформа Hagging Face: 
- класс Datasets используется для удобного, быстрого и совместимого для обучения инструментами HF набора данных для обучения
- использую преобученные bert эмбеддинги для обучения LSTM, для чего использую BertTokenizer из библиотеки Transformers. Данные эмбеддинги являются отличным вариантом для решения NLP задач, основанных на понимании сути текста
- в качестве контейнера для обучения объявленной модели я использую объект Trainer, который позволяет упростить технологию обучение модели (автоматически определяется оптимайзер, можно добавить конкретные метрики оценки качества обучения и визуализировать его, а также настроить автоматическое и условное сохранение оптимальных моделей)

В целом данный подход идентичен по сути традиционному обучению на голом PyTorch (например, проектирование модели абсолютно идентично), но гораздо более удобен и технологичен процесс обучения, оценки и сохранения модели

In [26]:
from datasets import load_dataset
from sklearn.preprocessing import LabelEncoder

### Шаг 1: Загрузка и предобработка данных с Hugging Face

Важно: сбор данных, подготовка датасета для задачи классификации и загрузка её на HF были сделаны на предыдущих этапах (смотри ноутбуки 1-2)

In [167]:
# Загрузка данных
dataset = load_dataset("data-silence/rus_news_classifier")

In [169]:
dataset

DatasetDict({
    train: Dataset({
        features: ['news', 'labels'],
        num_rows: 71913
    })
})

In [170]:
all_labels = dataset["train"]["labels"]

# Обучаем LabelEncoder на всех метках
label_encoder = LabelEncoder()
label_encoder.fit(all_labels)

LabelEncoder()

In [185]:
label_encoder.classes_

array(['climate', 'conflicts', 'culture', 'economy', 'gloss', 'health',
       'politics', 'science', 'society', 'sports', 'travel'], dtype='<U9')

In [186]:
cat_translator = dict()
for i, label in enumerate(label_encoder.classes_):
    cat_translator[label] = i

In [187]:
cat_translator

{'climate': 0,
 'conflicts': 1,
 'culture': 2,
 'economy': 3,
 'gloss': 4,
 'health': 5,
 'politics': 6,
 'science': 7,
 'society': 8,
 'sports': 9,
 'travel': 10}

In [171]:
# Преобразуем метки в закодированные значения
def encode_labels(batch):
    batch['labels'] = label_encoder.transform(batch['labels'])
    return batch

dataset = dataset.map(encode_labels, batched=True)

In [172]:
num_labels=len(label_encoder.classes_)
num_labels

11

In [174]:
dataset

DatasetDict({
    train: Dataset({
        features: ['news', 'labels'],
        num_rows: 71913
    })
})

In [173]:
dataset['train'][1]

{'news': 'Ретейлеры начали ввозить в Россию электронику по новой схеме — через серый или параллельный импорт. Одним из первых нововведение подтвердил «Связной», сообщает «Коммерсантъ». Речь идет о смартфонах, ноутбуках, смарт-часах компаний Samsung, Apple, а также игровых консолях PlayStation, Xbox и Nintendo. Всего ассортимент расширился более чем на 100 позиций. В «Связном» отметили, что товары, полученные по параллельному импорту, будут на 10-20 процентов дороже аналогов, ввезенных в Россию до начала спецоперации на Украине. По словам президента компании Алексея Вуколова, прежних запасов хватит еще на два-три месяца. Ритейлер добавил, что предоставляет на новые позиции официальную гарантию и заверил, что проблем с активацией не будет, поскольку они предназначены для стран Евразийского экономического союза (ЕАЭС). Ранее первый зампред комитета Госдумы по экономической политике Денис Кравченко пояснил, что параллельный импорт абсолютно легален и даже признается правилами Всемирной тор

In [175]:
dataset = dataset["train"].train_test_split(test_size=0.2)

In [176]:
train_dataset = dataset["train"]
test_dataset = dataset["test"]

In [177]:
train_dataset, test_dataset

(Dataset({
     features: ['news', 'labels'],
     num_rows: 57530
 }),
 Dataset({
     features: ['news', 'labels'],
     num_rows: 14383
 }))

In [180]:
from datasets import DatasetDict

dataset_dict = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

In [181]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['news', 'labels'],
        num_rows: 57530
    })
    test: Dataset({
        features: ['news', 'labels'],
        num_rows: 14383
    })
})

In [182]:
dataset_dict.push_to_hub("data-silence/rus_news_classifier")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/58 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/15 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/548 [00:00<?, ?B/s]

D:\OneDrive\IT\Projects\rewriter_projects\.venv\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\datasets--data-silence--rus_news_classifier. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


CommitInfo(commit_url='https://huggingface.co/datasets/data-silence/rus_news_classifier/commit/80bcc710b87f143a9329ef3cfff15c87d72bcfb2', commit_message='Upload dataset', commit_description='', oid='80bcc710b87f143a9329ef3cfff15c87d72bcfb2', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
train_dataset = dataset_dict['train']
test_dataset = dataset_dict['test']

### Шаг 2: Токенизация текстов

In [40]:
from transformers import BertTokenizer

# Загрузка токенизатора
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

def tokenize_function(example):
    return tokenizer(example["news"], padding="max_length", truncation=True)

# Применение токенизации
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

D:\OneDrive\IT\Projects\rewriter_projects\.venv\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--bert-base-multilingual-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

Map:   0%|          | 0/57530 [00:00<?, ? examples/s]

Map:   0%|          | 0/14383 [00:00<?, ? examples/s]

### Шаг 3: Построение модели на основе BiLSTM

In [43]:
num_labels

11

In [50]:
import torch
import torch.nn as nn
from transformers import BertModel

class BiLSTMClassifier(nn.Module):
    def __init__(self, hidden_dim, output_dim, n_layers, dropout):
        super(BiLSTMClassifier, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-multilingual-cased")
        self.lstm = nn.LSTM(self.bert.config.hidden_size, hidden_dim, num_layers=n_layers, 
                            bidirectional=True, dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input_ids, attention_mask, labels=None):
            with torch.no_grad():
                embedded = self.bert(input_ids=input_ids, attention_mask=attention_mask)[0]
            lstm_out, _ = self.lstm(embedded)
            pooled = torch.mean(lstm_out, dim=1)
            logits = self.fc(self.dropout(pooled))
            
            if labels is not None:
                loss_fn = nn.CrossEntropyLoss()
                loss = loss_fn(logits, labels)
                return {"loss": loss, "logits": logits}  # Возвращаем словарь
            return logits  # Возвращаем логиты, если метки не переданы

# Параметры модели
HIDDEN_DIM = 256
OUTPUT_DIM = num_labels  # Количество классов
N_LAYERS = 2
DROPOUT = 0.5

model = BiLSTMClassifier(HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, DROPOUT)


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

### Шаг 4: Обучение модели

In [56]:
from transformers import Trainer, TrainingArguments
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss

from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import EvalPrediction


def compute_metrics(p: EvalPrediction):
    preds = torch.argmax(p.predictions, axis=1)
    labels = p.label_ids

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }



# Параметры обучения
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

# Функция потерь и оптимизатор
criterion = CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=2e-5)

# Определение тренера
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, None)
)


In [55]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [57]:
# Перемещаем модель на GPU
model.to(device)

BiLSTMClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [58]:
# Запуск обучения
trainer.train()

Epoch,Training Loss,Validation Loss


TypeError: argmax(): argument 'input' (position 1) must be Tensor, not numpy.ndarray

Мы обучили модель на одной эпохе в течение 1,5 часов, и упали на этапе оценке модели, в связи с тем, что в функции compute_metrics не было предусмотрено преобразование логитов и меток истиных категорий в PyTorch тензор, а Trainer не умеет работать с numpy. Сохраним текущее состояние модели, исправим функцию, создадим новый объект Trainer и запустим его в режиме валидации, чтобы оценить текущее качество нашей модели

In [62]:
# Сохранение модели и оптимизатора
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'training_args': training_args,
}, "bilstm_model_after_first_epoch.pth")

### Шаг 5: Оценка качества модели

In [68]:
# Исправляем функкцию compute_metrics 
import torch
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    logits, labels = pred
    # Преобразуем logits и labels в тензоры PyTorch, если они являются массивами NumPy
    if isinstance(logits, np.ndarray):
        logits = torch.tensor(logits)
    if isinstance(labels, np.ndarray):
        labels = torch.tensor(labels)
    
    # Применяем операции только к тензорам PyTorch
    predictions = torch.argmax(logits, dim=-1).detach().cpu().numpy()
    labels = labels.detach().cpu().numpy()
    
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    
    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


In [71]:
# Опредяляем новый объект Trainer
new_trainer = Trainer(
    model=model,
    args=training_args,  # Существующие аргументы обучения
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics  # Новая функция compute_metrics
)

In [72]:
# Запускаем оценку текущего качества
new_trainer.evaluate()

{'eval_loss': 0.6787952780723572,
 'eval_model_preparation_time': 0.004,
 'eval_accuracy': 0.7777932281165264,
 'eval_f1': 0.7793803072537108,
 'eval_precision': 0.7893175071600528,
 'eval_recall': 0.7777932281165264,
 'eval_runtime': 1324.9069,
 'eval_samples_per_second': 10.856,
 'eval_steps_per_second': 0.679}

Качество на валидации в районе 78%, что не очень хорошо. Однако LSTM учится долго, я не стал доообучать ещё пару эпох, чтобы качество приблизилось к целевым 90%. Сохраним модель и токенайзер локально и загрузим их на HaggingFace

### Шаг 5: Сохранение обученной модели и токенайзера на HaggingFace

In [73]:
from huggingface_hub import HfApi, HfFolder

# Вход с использованием токена
HfFolder.save_token("hf_token")

In [124]:
tokenizer.save_pretrained('./')

In [127]:
# Создаем экземпляр конфигурации
new_config = BiLSTMConfig(
    vocab_size=tokenizer.vocab_size,
    hidden_size=256,
    num_layers=2,
    dropout=0.5,
    num_labels=11
)

# Инициализируем новую модель с конфигурацией
new_model = BiLSTMModel(new_config)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

D:\OneDrive\IT\Projects\rewriter_projects\.venv\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [143]:
tokenizer.push_to_hub('data-silence/lstm-news-classifier')

CommitInfo(commit_url='https://huggingface.co/data-silence/lstm-news-classifier/commit/aa4a79e84df969e75f1c57bc6c946138979a351a', commit_message='Upload tokenizer', commit_description='', oid='aa4a79e84df969e75f1c57bc6c946138979a351a', pr_url=None, pr_revision=None, pr_num=None)

In [148]:
torch.save(new_model, "model.pth")

In [153]:
new_model

__main__.BiLSTMClassifier

Механизм применения модели описан отдельным четверым ноутбуком в репозитории